FASE 1
1.-TEOREMA DE BAYES
Queremos la probabilidad de que una reseña sea positiva dado su texto "R"

                        P(A|R) = P(B|A) · P(A)
                                ---------------
                                      P(B)

Probabilidad a priori P(Positivo)
La “creencia inicial” de que una reseña sea positiva antes de leerla.
En tu dataset se estima con entrenamiento:

                P(Positivo)     = reseñas positivas en train(80%)  
                                -----------------------------------
                                      reseñas totales en train

                P(Negativo) = 1 - P(Positivo)

VEROSIMILITUD P(R|Positivo)
Con naive Bayes miltinomial (bolsa de palabras positivas y negativas con conteos)

    Cada P(w|Positivo) se estima con Laplace

                P(w|Pos) = count(P(positivos)(w)) + 1 
                           --------------------------
                                T(P(positivos))+V

para una reseña  𝑅 y una clase  𝑐 (positivo o negativo), la verosimilitud 𝑝(𝑅∣𝑐) mide qué tan probable es “generar” ese texto si la clase fuera 𝑐. en el modelo multinomial de bolsa de palabras, representamos 𝑅  por los conteos  𝑛𝑤 de cada palabra  𝑤. con la suposición ingenua (ver abajo), tratamos las palabras como independientes condicionadas a la clase, así que la verosimilitud se descompone en un producto de probabilidades palabra–clase: “cuántas veces aparece cada palabra y qué tan típica es de la clase”.


suposición ingenua (naive).
se llama “ingenua” porque asume que las palabras de la reseña son independientes entre sí dado la clase. esto ignora orden, gramática y dependencias (por ejemplo, la interacción de “no” con “buena”), pero hace el cálculo simple y eficiente. a pesar de ser una aproximación grosera, funciona bien en tareas como clasificación de reseñas.

qué queda fuera (evidencia) y cómo se decide.
la evidencia 𝑝(𝑅) es un factor de normalización común a todas las clases; para clasificar no hace falta calcularla: se comparan los “scores” por clase
score(c) = logp(c) + SUMATORIA(w,)n(w) logp(w|c)
y se elige la clase con mayor score. aquí logp(c) es la probabilidad a priori (qué fracción de reseñas del train son positivas/negativas) y la suma representa la verosimilitud en log con laplace.

In [1]:
import pandas as pd
from collections import Counter, defaultdict
import re
import math

#FASE 2 PREPARACION Y LIMPIEZA DE DATOS
#yo pongo todo dentro de funciones para usarlas luego;p
TEXT_COL = "review_es"
LABEL_COL = "sentimiento" #cambiamos positive/negative por

#mis stopwords owowowowo
STOP = {
    'el','la','los','las','de','del','y','o','u','que','a','un','una','en','es','se',
    'lo','por','con','al','como','para','su','sus','le','les','me','mi','tu','te','ya','muy',
    'pero','si','sí','mas','más','cuando','donde','dónde'
}

#definimos nuestra funcion parar separar palabras, quitar stopwords y espacios extras
def barrer_y_botar(texto: str, quitar_stops=True):
    t = str(texto).lower()#minusculas
    t = re.sub(r"[^a-záéíóúüñ\s]", " ",t) #quitamos char raros o el espacio
    t = re.sub(r"\s+", " ",t).strip() #quitamos espacios extras
#TOKENICE
    tokens = t.split()
    if quitar_stops:
        tokens = [w for w in tokens if w not in STOP]
    return tokens

In [2]:
#cargamos datos
df = pd.read_csv("IMDB Dataset SPANISH.csv")
df["tokens"]= df["review_es"].apply(lambda x: barrer_y_botar(x)) #recorremnos ada reseña y aplicamos la funcion d limpieza
print(len(df),"reseñas")
print(df["tokens"].iloc[0][:20])

50000 reseñas
['uno', 'otros', 'críticos', 'ha', 'mencionado', 'después', 'ver', 'solo', 'oz', 'episodio', 'estará', 'enganchado', 'tienen', 'razón', 'esto', 'exactamente', 'sucedió', 'conmigo', 'primera', 'cosa']


In [13]:
#FASE 3 ENTRENAMIENTO
#comenzamos la mescla de datos y el 80/20 (Trabajaremos con el set de entrenamiento "train")
df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)
cut = int(0.8 * len(df)) #40000
train = df.iloc[0:cut].copy()
test = df.iloc[cut:].copy()

train["y"] = train["sentimiento"].map({"positivo":"pos", "negativo":"neg"})
test["y"] = test["sentimiento"].map({"positivo":"pos","negativo":"neg"})

N = len(train) #40000
Npos = (train["y"] == "pos").sum()
Nneg = (train["y"] == "neg").sum()
#calcular Probabilidades A Priori (P(A)):
P_pos = Npos / N
P_neg = Nneg / N
print(f"train: {N}  | pos: {Npos}  neg: {Nneg}")
print(f"P(Pos)={P_pos:.4f}  P(Neg)={P_neg:.4f}")

train: 40000  | pos: 20042  neg: 19958
P(Pos)=0.5010  P(Neg)=0.4990


In [14]:
# 2.CONSTRUIR EL VOCABULARIO
#creamos dos bolsas d palabras, para contar ocurrencias de positivas y negativas
bolsa_pos = Counter()
bolsa_neg = Counter()

#Contamos el numero total de palabras en reseñas posi y negativas
for toks, y in zip(train["tokens"], train["y"]):
    if y == "pos":
        bolsa_pos.update(toks)
    else:
        bolsa_neg.update(toks)
tot_pos = sum(bolsa_pos.values())
tot_neg = sum(bolsa_neg.values())

vocab = set(bolsa_pos) | set(bolsa_neg)
V = len(vocab)
print("tot_pos:", tot_pos, " | tot_neg:", tot_neg, " | Vocab size:", V)


tot_pos: 3026354  | tot_neg: 2988159  | Vocab size: 155590


In [15]:
#3 Calculamos probabilidades condicionales
alpha = 1.0 #suavizado de Laplace
den_pos = tot_pos + alpha * V
den_neg = tot_neg + alpha * V

#tablas de probabilidades para cada palabra
logP_w_pos = {}
logP_w_neg = {}
for w in vocab:
    logP_w_pos[w] = math.log((bolsa_pos[w] + alpha) / den_pos)
    logP_w_neg[w] = math.log((bolsa_neg[w] + alpha) / den_neg)

# Probabilidades a priori en log
logprior_pos = math.log(P_pos)
logprior_neg = math.log(P_neg)

# Valor por defecto para palabras desconocidas
default_pos = math.log(alpha / den_pos)
default_neg = math.log(alpha / den_neg)

def clasificar(tokens):
    score_pos = logprior_pos
    score_neg = logprior_neg
    for w in tokens:
        score_pos += logP_w_pos.get(w, default_pos)
        score_neg += logP_w_neg.get(w, default_neg)
    return ('pos' if score_pos > score_neg else 'neg'), score_pos, score_neg


In [16]:
def ver_prob_palabra(w):
    # conteo crudos
    c_pos = bolsa_pos[w]
    c_neg = bolsa_neg[w]
    print(f"conteos -> pos:{c_pos}  neg:{c_neg}")

    # sin suavizado puede dar 0
    p_pos_naive = c_pos / tot_pos if tot_pos else 0.0
    p_neg_naive = c_neg / tot_neg if tot_neg else 0.0
    print(f"sin suavizado -> P({w}|pos)={p_pos_naive:.3e}  P({w}|neg)={p_neg_naive:.3e}")

    # con Laplace (α=1)
    p_pos_lap = (c_pos + 1) / den_pos
    p_neg_lap = (c_neg + 1) / den_neg
    print(f"con Laplace(α=1) -> P({w}|pos)={p_pos_lap:.3e}  P({w}|neg)={p_neg_lap:.3e}")

# ejemplo con la palabra que preguntas
w = "fantástica" #la limpieza mantiene acentos y minúsculas
ver_prob_palabra(w)

conteos -> pos:426  neg:96
sin suavizado -> P(fantástica|pos)=1.408e-04  P(fantástica|neg)=3.213e-05
con Laplace(α=1) -> P(fantástica|pos)=1.342e-04  P(fantástica|neg)=3.085e-05


In [17]:
# P(palabra|Positivo)
rows = []
for w in vocab:
    cpos = bolsa_pos[w]
    cneg = bolsa_neg[w]
    ppos = (cpos + alpha) / den_pos
    pneg = (cneg + alpha) / den_neg
    rows.append((w, cpos, cneg, ppos, pneg, math.log(ppos), math.log(pneg)))

probs_df = pd.DataFrame(rows, columns=[
    'word','count_pos','count_neg','P_pos','P_neg','logP_w_pos','logP_w_neg'
])
pd.options.display.float_format = '{:.12f}'.format  # 12 decimales
probs_df.head(V)

,word,count_pos,count_neg,P_pos,P_neg,logP_w_pos,logP_w_neg
0,receptionist,1,0,0.000000628547,0.000000318092,-14.279855708171,-14.960926594739
1,tugurio,3,0,0.000001257093,0.000000318092,-13.586708527611,-14.960926594739
2,recorre,4,4,0.000001571366,0.000001590458,-13.363564976297,-13.351488682305
3,mauritz,0,1,0.000000314273,0.000000636183,-14.973002888731,-14.267779414179
4,tópicamente,1,0,0.000000628547,0.000000318092,-14.279855708171,-14.960926594739
...,...,...,...,...,...,...,...
155585,atribuyen,1,1,0.000000628547,0.000000636183,-14.279855708171,-14.267779414179
155586,efeminante,0,1,0.000000314273,0.000000636183,-14.973002888731,-14.267779414179
155587,aeróbicas,0,1,0.000000314273,0.000000636183,-14.973002888731,-14.267779414179
155588,klebold,1,1,0.000000628547,0.000000636183,-14.279855708171,-14.267779414179


In [18]:
#verificamos que todo este bien, cada uno deberia sumar 1 por clase
print("Suma P(w|pos) ≈", probs_df['P_pos'].sum())
print("Suma P(w|neg) ≈", probs_df['P_neg'].sum())

Suma P(w|pos) ≈ 1.0
Suma P(w|neg) ≈ 1.0


In [19]:
#Fase 4 CLASIFICACION (APLICACION DE BAYES)
def clasificar_sentimiento(reseña):
    # 1. limpiar y tokenizar igual que en Fase 2
    tokens = barrer_y_botar(reseña)

    # 2. iniciar los puntajes con los priors en log
    score_pos = logprior_pos  # log(P(Positivo))
    score_neg = logprior_neg  # log(P(Negativo))

    # 3. sumar los log-probs de cada palabra según cada clase
    for w in tokens:
        score_pos += logP_w_pos.get(w, default_pos)  # log(P(palabra|Positivo))
        score_neg += logP_w_neg.get(w, default_neg)  # log(P(palabra|Negativo))
#usamos los logaritmos pq si multiplicaramos todas las probabiliadades quedaria algo como:
# 0.00002 * 0.000001 * 0.0005 * ... = 0.000000000000000000000000000000...
#y eso se aproxima a 0 = underflow aritmetico, el modelo explotaria
#en cambio con los logaritmos transforman multiplicaciones en sumas
    # 4. decidir clase
    if score_pos > score_neg:
        return "pos"
    else:
        return "neg"


In [20]:
#Fase 5
#aqui estamos trabajando con el 20%, pq usamos el 80% ya para entrenar
# hacemos copia por seguridad
test = test.copy()

# predecimos cada reseña del set de prueba
test["prediccion"] = test["review_es"].apply(clasificar_sentimiento)

# accuracy
correctas = (test["prediccion"] == test["y"]).sum()
total = len(test)
accuracy = correctas / total

print(f"Reseñas correctas: {correctas} / {total}")
print(f"Exactitud (accuracy): {accuracy:.3f}")

Reseñas correctas: 8385 / 10000
Exactitud (accuracy): 0.839


¿Que exactitud obtuvo el modelo? ¿Consideran que es un buen resultado?
la exactitud fue del 0.835 (8346/10000), si es buen resultado, 83-84% es muy solido y muy por encima del azar

In [21]:
# ordenar por prob condicional directa
top_pos_palabras = probs_df.sort_values("P_pos", ascending=False).head(10)
top_neg_palabras = probs_df.sort_values("P_neg", ascending=False).head(10)

print("Top 10 palabras más probables en reseñas POSITIVAS:")
print(top_pos_palabras[["word", "P_pos", "count_pos", "count_neg"]].to_string(index=False))

print("\nTop 10 palabras más probables en reseñas NEGATIVAS:")
print(top_neg_palabras[["word", "P_neg", "count_neg", "count_pos"]].to_string(index=False))


Top 10 palabras más probables en reseñas POSITIVAS:
    word          P_pos  count_pos  count_neg
película 0.018130425928      57689      63870
      no 0.015402219524      49008      67206
    esta 0.009695645178      30850      34531
     the 0.007331052966      23326      19061
    está 0.004684871890      14906      15134
     fue 0.004115408694      13094      14796
    este 0.003691454029      11745      13159
historia 0.003654998328      11629       9177
     son 0.003597800590      11447      11288
     and 0.003556316516      11315       8009

Top 10 palabras más probables en reseñas NEGATIVAS:
    word          P_neg  count_neg  count_pos
      no 0.021377978967      67206      49008
película 0.020316825548      63870      57689
    esta 0.010984337490      34531      30850
     the 0.006063461173      19061      23326
    está 0.004814315647      15134      14906
     fue 0.004706800702      14796      13094
    este 0.004186084831      13159      11745
    solo 0.0041135599

¿Qué muestran las listas de palabras?
Reflejan frecuencia, no necesariamente poder discriminativo, palabras como "pelicula", "esta" "fue/este" aparecen en alto em ambas clases, por que son comunes en todas las reseñas
Tambien podemos ver que la negacion "No" es mas probable en negativas, el modelo si capto eso

In [22]:
resena_pos_ejemplo = "Disfrute mucho de la película, me gusto"
resena_neg_ejemplo = "Esta película no fue de mi gusto, the"

print("Reseña positiva ejemplo:")
print(resena_pos_ejemplo)
print("Predicción:", clasificar_sentimiento(resena_pos_ejemplo))

print("\nReseña negativa ejemplo:")
print(resena_neg_ejemplo)
print("Predicción:", clasificar_sentimiento(resena_neg_ejemplo))


Reseña positiva ejemplo:
Disfrute mucho de la película, me gusto
Predicción: pos

Reseña negativa ejemplo:
Esta película no fue de mi gusto, the
Predicción: neg
